In [1]:
import pandas as pd
import numpy as np
import random
np.random.seed(0)

In [2]:
# Get data

df_papers = pd.read_csv('../konok_data/papers_data.csv')
df_papers.loc[df_papers['field_query'] == ' Big Data', 'field_query'] = 'Big Data' # Fixed Big Data spacing issue
df_papers.head()

,field_query,paperId,title,url,venue,publicationTypes,abstract,year,citationCount,journal_name,journal_volume,journal_pages
0,NLP,29ddc1f43f28af7c846515e32cc167bc66886d0c,Parameter-Efficient Transfer Learning for NLP,https://www.semanticscholar.org/paper/29ddc1f4...,International Conference on Machine Learning,"['JournalArticle', 'Conference']",Fine-tuning large pre-trained models is an eff...,2019,2453,ArXiv,abs/1902.00751,NaN
1,NLP,58ed1fbaabe027345f7bb3a6312d41c5aac63e22,Retrieval-Augmented Generation for Knowledge-I...,https://www.semanticscholar.org/paper/58ed1fba...,Neural Information Processing Systems,['JournalArticle'],Large pre-trained language models have been sh...,2020,1686,ArXiv,abs/2005.11401,NaN
2,NLP,d6a083dad7114f3a39adc65c09bfbb6cf3fee9ea,Energy and Policy Considerations for Deep Lear...,https://www.semanticscholar.org/paper/d6a083da...,Annual Meeting of the Association for Computat...,"['JournalArticle', 'Conference']",Recent progress in hardware and methodology fo...,2019,2113,ArXiv,abs/1906.02243,NaN
3,NLP,06d7cb8c8816360feb33c3367073e0ef66d7d0b0,Super-NaturalInstructions: Generalization via ...,https://www.semanticscholar.org/paper/06d7cb8c...,Conference on Empirical Methods in Natural Lan...,"['JournalArticle', 'Conference']",How well can NLP models generalize to a variet...,2022,432,NaN,NaN,5085-5109
4,NLP,5471114e37448bea2457b74894b1ecb92bbcfdf6,From Pretraining Data to Language Models to Do...,https://www.semanticscholar.org/paper/5471114e...,Annual Meeting of the Association for Computat...,"['JournalArticle', 'Conference']",Language models (LMs) are pretrained on divers...,2023,72,ArXiv,abs/2305.08283,NaN


In [3]:
df_papers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   field_query       1200 non-null   object
 1   paperId           1200 non-null   object
 2   title             1200 non-null   object
 3   url               1200 non-null   object
 4   venue             1122 non-null   object
 5   publicationTypes  1034 non-null   object
 6   abstract          892 non-null    object
 7   year              1200 non-null   int64 
 8   citationCount     1200 non-null   int64 
 9   journal_name      1038 non-null   object
 10  journal_volume    776 non-null    object
 11  journal_pages     645 non-null    object
dtypes: int64(2), object(10)
memory usage: 112.6+ KB


In [4]:
df_papers['field_query'].value_counts()

NLP                  100
Machine Learning     100
LLM                  100
Deep Learning        100
Quantum Computing    100
Graph Database       100
Data Management      100
Data Modeling        100
Big Data             100
Data Processing      100
Data Storage         100
Data Querying        100
Name: field_query, dtype: int64

# Authors & Corresponding Author

In [5]:
df_authors = pd.read_csv('../konok_data/authors_data.csv')
df_authors = df_authors.dropna(subset=['authorId'])
df_authors.head()

,paperId,authorId,name,affiliations
0,29ddc1f43f28af7c846515e32cc167bc66886d0c,2815290.0,N. Houlsby,NaN
1,29ddc1f43f28af7c846515e32cc167bc66886d0c,1911881.0,A. Giurgiu,NaN
2,29ddc1f43f28af7c846515e32cc167bc66886d0c,40569328.0,Stanislaw Jastrzebski,NaN
3,29ddc1f43f28af7c846515e32cc167bc66886d0c,68973833.0,Bruna Morrone,NaN
4,29ddc1f43f28af7c846515e32cc167bc66886d0c,51985388.0,Quentin de Laroussilhe,NaN


In [6]:
df_author_paper = df_authors.groupby('paperId')['authorId'].agg(list).reset_index()
df_author_paper.head()

,paperId,authorId
0,0026ea8a0fd31bdc959a4b9ed4d449f3015be8d1,"[71940750.0, 1657644010.0, 48633912.0, 9042677..."
1,003ef1cd670d01af05afa0d3c72d72228f494432,"[145306564.0, 48751979.0, 2682457.0, 215519324..."
2,0084f3cb0a1754272151c5268a783f24bf5676a0,"[2281247548.0, 1754598.0, 9379106.0, 140926783..."
3,0090023afc66cd2741568599057f4e82b566137c,"[51997673.0, 2775559.0, 51884035.0, 1782658.0,..."
4,0095acc4f2c3255cf38fdf844003c97858adb418,"[2138646471.0, 143655216.0, 1764004.0]"


In [7]:
len(df_author_paper)

1156

In [8]:
df_papers = pd.merge(df_papers, df_author_paper, on='paperId', how='left')
df_papers.rename(columns={'authorId': 'authorIds'}, inplace=True)
df_papers = df_papers.dropna(subset=['authorIds'])
df_papers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1164 entries, 0 to 1199
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   field_query       1164 non-null   object
 1   paperId           1164 non-null   object
 2   title             1164 non-null   object
 3   url               1164 non-null   object
 4   venue             1091 non-null   object
 5   publicationTypes  1012 non-null   object
 6   abstract          873 non-null    object
 7   year              1164 non-null   int64 
 8   citationCount     1164 non-null   int64 
 9   journal_name      1006 non-null   object
 10  journal_volume    759 non-null    object
 11  journal_pages     633 non-null    object
 12  authorIds         1164 non-null   object
dtypes: int64(2), object(11)
memory usage: 127.3+ KB


In [9]:
# corresponding author is the main author
df_papers['corresponding_author'] = df_papers['authorIds'].apply(lambda x: x[0])
df_papers.head()

,field_query,paperId,title,url,venue,publicationTypes,abstract,year,citationCount,journal_name,journal_volume,journal_pages,authorIds,corresponding_author
0,NLP,29ddc1f43f28af7c846515e32cc167bc66886d0c,Parameter-Efficient Transfer Learning for NLP,https://www.semanticscholar.org/paper/29ddc1f4...,International Conference on Machine Learning,"['JournalArticle', 'Conference']",Fine-tuning large pre-trained models is an eff...,2019,2453,ArXiv,abs/1902.00751,NaN,"[2815290.0, 1911881.0, 40569328.0, 68973833.0,...",2.815290e+06
1,NLP,58ed1fbaabe027345f7bb3a6312d41c5aac63e22,Retrieval-Augmented Generation for Knowledge-I...,https://www.semanticscholar.org/paper/58ed1fba...,Neural Information Processing Systems,['JournalArticle'],Large pre-trained language models have been sh...,2020,1686,ArXiv,abs/2005.11401,NaN,"[145222654.0, 3439053.0, 1716179427.0, 4005230...",1.452227e+08
2,NLP,d6a083dad7114f3a39adc65c09bfbb6cf3fee9ea,Energy and Policy Considerations for Deep Lear...,https://www.semanticscholar.org/paper/d6a083da...,Annual Meeting of the Association for Computat...,"['JournalArticle', 'Conference']",Recent progress in hardware and methodology fo...,2019,2113,ArXiv,abs/1906.02243,NaN,"[2268272.0, 47079359.0, 143753639.0]",2.268272e+06
3,NLP,06d7cb8c8816360feb33c3367073e0ef66d7d0b0,Super-NaturalInstructions: Generalization via ...,https://www.semanticscholar.org/paper/06d7cb8c...,Conference on Empirical Methods in Natural Lan...,"['JournalArticle', 'Conference']",How well can NLP models generalize to a variet...,2022,432,NaN,NaN,5085-5109,"[1705260.0, 1817207.0, 1805993128.0, 215653883...",1.705260e+06
4,NLP,5471114e37448bea2457b74894b1ecb92bbcfdf6,From Pretraining Data to Language Models to Do...,https://www.semanticscholar.org/paper/5471114e...,Annual Meeting of the Association for Computat...,"['JournalArticle', 'Conference']",Language models (LMs) are pretrained on divers...,2023,72,ArXiv,abs/2305.08283,NaN,"[2114887261.0, 50487261.0, 2169159066.0, 20735...",2.114887e+09


# Cleanup Data

Based on this, venue seems useless. Therefore, I will remove these datapoints.Similarily remove other stuff as well. Keep only useful and complete stuff for now.

In [10]:
df_papers = df_papers.dropna(subset=['venue'])
df_papers = df_papers.dropna(subset=['publicationTypes'])
df_papers = df_papers.dropna(subset=['abstract'])
df_papers = df_papers.drop(columns=['journal_pages']) # its not needed

df_papers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 757 entries, 0 to 1199
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   field_query           757 non-null    object 
 1   paperId               757 non-null    object 
 2   title                 757 non-null    object 
 3   url                   757 non-null    object 
 4   venue                 757 non-null    object 
 5   publicationTypes      757 non-null    object 
 6   abstract              757 non-null    object 
 7   year                  757 non-null    int64  
 8   citationCount         757 non-null    int64  
 9   journal_name          672 non-null    object 
 10  journal_volume        506 non-null    object 
 11  authorIds             757 non-null    object 
 12  corresponding_author  757 non-null    float64
dtypes: float64(1), int64(2), object(10)
memory usage: 82.8+ KB


In [11]:
# Filter rows where 'venue' contains 'Workshop' after filling NaN values
workshop_rows = df_papers[df_papers['venue'].fillna('').str.contains('Workshop')]

# Change the value of 'publicationTypes' to 'Workshop' for the filtered rows
df_papers.loc[workshop_rows.index, 'publicationTypes'] = "['Workshop']"

In [12]:
# Filter rows where 'publicationTypes' contains 'Conference' after filling NaN values
conference_rows = df_papers[df_papers['publicationTypes'].fillna('').str.contains('Conference')]

# Change the value of 'publicationTypes' to 'Conference' for the filtered rows
df_papers.loc[conference_rows.index, 'publicationTypes'] = "['Conference']"

In [13]:
df_papers['publicationTypes'].value_counts()

['JournalArticle']                      366
['Conference']                          183
['JournalArticle', 'Review']            120
['Review']                               35
['Review', 'JournalArticle']             23
['Workshop']                             11
['JournalArticle', 'Book']               10
['Book', 'JournalArticle']                4
['Book', 'JournalArticle', 'Review']      1
['Book']                                  1
['Study', 'JournalArticle']               1
['JournalArticle', 'Study']               1
['CaseReport', 'Review']                  1
Name: publicationTypes, dtype: int64

In [14]:
# Only Keep the three for now

allowed_values = ["['JournalArticle']", "['Conference']", "['Workshop']"]
df_papers = df_papers[df_papers['publicationTypes'].isin(allowed_values)]

df_papers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 560 entries, 0 to 1198
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   field_query           560 non-null    object 
 1   paperId               560 non-null    object 
 2   title                 560 non-null    object 
 3   url                   560 non-null    object 
 4   venue                 560 non-null    object 
 5   publicationTypes      560 non-null    object 
 6   abstract              560 non-null    object 
 7   year                  560 non-null    int64  
 8   citationCount         560 non-null    int64  
 9   journal_name          477 non-null    object 
 10  journal_volume        361 non-null    object 
 11  authorIds             560 non-null    object 
 12  corresponding_author  560 non-null    float64
dtypes: float64(1), int64(2), object(10)
memory usage: 61.2+ KB


In [15]:
df_papers['publicationTypes'] = df_papers['publicationTypes'].str[2:-2]
df_papers.head()

,field_query,paperId,title,url,venue,publicationTypes,abstract,year,citationCount,journal_name,journal_volume,authorIds,corresponding_author
0,NLP,29ddc1f43f28af7c846515e32cc167bc66886d0c,Parameter-Efficient Transfer Learning for NLP,https://www.semanticscholar.org/paper/29ddc1f4...,International Conference on Machine Learning,Conference,Fine-tuning large pre-trained models is an eff...,2019,2453,ArXiv,abs/1902.00751,"[2815290.0, 1911881.0, 40569328.0, 68973833.0,...",2.815290e+06
1,NLP,58ed1fbaabe027345f7bb3a6312d41c5aac63e22,Retrieval-Augmented Generation for Knowledge-I...,https://www.semanticscholar.org/paper/58ed1fba...,Neural Information Processing Systems,JournalArticle,Large pre-trained language models have been sh...,2020,1686,ArXiv,abs/2005.11401,"[145222654.0, 3439053.0, 1716179427.0, 4005230...",1.452227e+08
2,NLP,d6a083dad7114f3a39adc65c09bfbb6cf3fee9ea,Energy and Policy Considerations for Deep Lear...,https://www.semanticscholar.org/paper/d6a083da...,Annual Meeting of the Association for Computat...,Conference,Recent progress in hardware and methodology fo...,2019,2113,ArXiv,abs/1906.02243,"[2268272.0, 47079359.0, 143753639.0]",2.268272e+06
3,NLP,06d7cb8c8816360feb33c3367073e0ef66d7d0b0,Super-NaturalInstructions: Generalization via ...,https://www.semanticscholar.org/paper/06d7cb8c...,Conference on Empirical Methods in Natural Lan...,Conference,How well can NLP models generalize to a variet...,2022,432,NaN,NaN,"[1705260.0, 1817207.0, 1805993128.0, 215653883...",1.705260e+06
4,NLP,5471114e37448bea2457b74894b1ecb92bbcfdf6,From Pretraining Data to Language Models to Do...,https://www.semanticscholar.org/paper/5471114e...,Annual Meeting of the Association for Computat...,Conference,Language models (LMs) are pretrained on divers...,2023,72,ArXiv,abs/2305.08283,"[2114887261.0, 50487261.0, 2169159066.0, 20735...",2.114887e+09


In [16]:
df_papers['publicationTypes'].value_counts()

JournalArticle    366
Conference        183
Workshop           11
Name: publicationTypes, dtype: int64

In [17]:
df_papers['journal_name']

0                                                   ArXiv
1                                                   ArXiv
2                                                   ArXiv
3                                                     NaN
4                                                   ArXiv
                              ...                        
1191    Proceedings of the 2016 International Conferen...
1194    Proceedings of the 2017 ACM International Conf...
1195                               Nucleic Acids Research
1197                                                  NaN
1198    2017 IEEE 33rd International Conference on Dat...
Name: journal_name, Length: 560, dtype: object

In [18]:
# Working on journal_volume

# Identify rows where 'publicationTypes' is either 'Workshop' or 'Conference'
mask = df_papers['publicationTypes'].isin(['Workshop', 'Conference'])

# Set corresponding values in 'journal_volume' to an empty string
df_papers.loc[mask, 'journal_volume'] = ""

In [19]:
df_papers['journal_volume'].isna().sum()

43

In [20]:
# implies there are NaN values for Journals too. So, we need to remove them.
df_papers = df_papers.dropna(subset=['journal_volume'])
df_papers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 517 entries, 0 to 1198
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   field_query           517 non-null    object 
 1   paperId               517 non-null    object 
 2   title                 517 non-null    object 
 3   url                   517 non-null    object 
 4   venue                 517 non-null    object 
 5   publicationTypes      517 non-null    object 
 6   abstract              517 non-null    object 
 7   year                  517 non-null    int64  
 8   citationCount         517 non-null    int64  
 9   journal_name          450 non-null    object 
 10  journal_volume        517 non-null    object 
 11  authorIds             517 non-null    object 
 12  corresponding_author  517 non-null    float64
dtypes: float64(1), int64(2), object(10)
memory usage: 56.5+ KB


In [21]:
# check if all journals have journal names or not
df_papers.query('publicationTypes == "JournalArticle"')['journal_volume'].isna().sum()

0

Note : Currently the following information is missing 

- Edition (19th etc)
- Venue should be the location
- Journal name would be changed to publication type name (basically the stuff inside venue)

For Evolution part, I have to add the following : 
- Reviewer
- Comments
- Approved or Not

In [22]:
# Add Location

df_cities = pd.read_csv('../aryan_data/europe-capital-cities.csv')
capitals = df_cities['capital'].tolist()
df_papers['location'] = [random.choice(capitals) for _ in range(len(df_papers))]
df_papers.head()

,field_query,paperId,title,url,venue,publicationTypes,abstract,year,citationCount,journal_name,journal_volume,authorIds,corresponding_author,location
0,NLP,29ddc1f43f28af7c846515e32cc167bc66886d0c,Parameter-Efficient Transfer Learning for NLP,https://www.semanticscholar.org/paper/29ddc1f4...,International Conference on Machine Learning,Conference,Fine-tuning large pre-trained models is an eff...,2019,2453,ArXiv,,"[2815290.0, 1911881.0, 40569328.0, 68973833.0,...",2.815290e+06,Stockholm
1,NLP,58ed1fbaabe027345f7bb3a6312d41c5aac63e22,Retrieval-Augmented Generation for Knowledge-I...,https://www.semanticscholar.org/paper/58ed1fba...,Neural Information Processing Systems,JournalArticle,Large pre-trained language models have been sh...,2020,1686,ArXiv,abs/2005.11401,"[145222654.0, 3439053.0, 1716179427.0, 4005230...",1.452227e+08,Valletta
2,NLP,d6a083dad7114f3a39adc65c09bfbb6cf3fee9ea,Energy and Policy Considerations for Deep Lear...,https://www.semanticscholar.org/paper/d6a083da...,Annual Meeting of the Association for Computat...,Conference,Recent progress in hardware and methodology fo...,2019,2113,ArXiv,,"[2268272.0, 47079359.0, 143753639.0]",2.268272e+06,Stockholm
3,NLP,06d7cb8c8816360feb33c3367073e0ef66d7d0b0,Super-NaturalInstructions: Generalization via ...,https://www.semanticscholar.org/paper/06d7cb8c...,Conference on Empirical Methods in Natural Lan...,Conference,How well can NLP models generalize to a variet...,2022,432,NaN,,"[1705260.0, 1817207.0, 1805993128.0, 215653883...",1.705260e+06,San Marino
4,NLP,5471114e37448bea2457b74894b1ecb92bbcfdf6,From Pretraining Data to Language Models to Do...,https://www.semanticscholar.org/paper/5471114e...,Annual Meeting of the Association for Computat...,Conference,Language models (LMs) are pretrained on divers...,2023,72,ArXiv,,"[2114887261.0, 50487261.0, 2169159066.0, 20735...",2.114887e+09,Tallinn


In [23]:
# Function to select reviewer IDs for each row
def select_reviewers(row):
    author_ids = row['authorIds']
    reviewers = []
    
    # Generate a list of all author IDs from the 'authorIds' column
    all_author_ids = set(df_papers['authorIds'].explode())
    
    # Exclude the author IDs of the current row
    potential_reviewers = list(all_author_ids - set(author_ids))
    
    # Randomly select reviewers, excluding empty reviewer lists
    if potential_reviewers:
        reviewers = random.sample(potential_reviewers, min(3, len(potential_reviewers)))
        
    if any(reviewer_id in author_ids for reviewer_id in reviewers):
        print("Warning: Reviewer ID overlaps with Author ID for row with paperId:", row['paperId'])
    
    return reviewers

# Create a new column 'reviewers' containing a list of 3 reviewer IDs for each row
df_papers['reviewers'] = df_papers.apply(select_reviewers, axis=1)

In [24]:
df_papers[['reviewers', 'authorIds']]

,reviewers,authorIds
0,"[145222654.0, 2056908.0, 1398327640.0]","[2815290.0, 1911881.0, 40569328.0, 68973833.0,..."
1,"[50064714.0, 1877327478.0, 8579299.0]","[145222654.0, 3439053.0, 1716179427.0, 4005230..."
2,"[2113922820.0, 2176030144.0, 51352814.0]","[2268272.0, 47079359.0, 143753639.0]"
3,"[145507437.0, 144310754.0, 2113922820.0]","[1705260.0, 1817207.0, 1805993128.0, 215653883..."
4,"[2235966.0, 8491577.0, 47610941.0]","[2114887261.0, 50487261.0, 2169159066.0, 20735..."
...,...,...
1191,"[1576655836.0, 31140405.0, 48455738.0]","[2149234208.0, 1699363.0, 2186806073.0]"
1194,"[1741623.0, 1939292.0, 7478280.0]","[1730344.0, 1891554.0, 2128664093.0]"
1195,"[2057440722.0, 2118511088.0, 2108438707.0]","[2485039.0, 2039003.0, 2804763.0, 6562624.0, 4..."
1197,"[66395694.0, 2054594326.0, 2211964951.0]","[40275640.0, 1736279.0]"


In [25]:
# Paper is published or accepted, means that it is approved

df_papers['approved'] = 'Y'

In [35]:
# reviewer comments

df_reviews = pd.read_csv('../aryan_data/reviewer_comments.csv')

In [26]:
df_papers.head()

,field_query,paperId,title,url,venue,publicationTypes,abstract,year,citationCount,journal_name,journal_volume,authorIds,corresponding_author,location,reviewers,approved
0,NLP,29ddc1f43f28af7c846515e32cc167bc66886d0c,Parameter-Efficient Transfer Learning for NLP,https://www.semanticscholar.org/paper/29ddc1f4...,International Conference on Machine Learning,Conference,Fine-tuning large pre-trained models is an eff...,2019,2453,ArXiv,,"[2815290.0, 1911881.0, 40569328.0, 68973833.0,...",2.815290e+06,Stockholm,"[145222654.0, 2056908.0, 1398327640.0]",Y
1,NLP,58ed1fbaabe027345f7bb3a6312d41c5aac63e22,Retrieval-Augmented Generation for Knowledge-I...,https://www.semanticscholar.org/paper/58ed1fba...,Neural Information Processing Systems,JournalArticle,Large pre-trained language models have been sh...,2020,1686,ArXiv,abs/2005.11401,"[145222654.0, 3439053.0, 1716179427.0, 4005230...",1.452227e+08,Valletta,"[50064714.0, 1877327478.0, 8579299.0]",Y
2,NLP,d6a083dad7114f3a39adc65c09bfbb6cf3fee9ea,Energy and Policy Considerations for Deep Lear...,https://www.semanticscholar.org/paper/d6a083da...,Annual Meeting of the Association for Computat...,Conference,Recent progress in hardware and methodology fo...,2019,2113,ArXiv,,"[2268272.0, 47079359.0, 143753639.0]",2.268272e+06,Stockholm,"[2113922820.0, 2176030144.0, 51352814.0]",Y
3,NLP,06d7cb8c8816360feb33c3367073e0ef66d7d0b0,Super-NaturalInstructions: Generalization via ...,https://www.semanticscholar.org/paper/06d7cb8c...,Conference on Empirical Methods in Natural Lan...,Conference,How well can NLP models generalize to a variet...,2022,432,NaN,,"[1705260.0, 1817207.0, 1805993128.0, 215653883...",1.705260e+06,San Marino,"[145507437.0, 144310754.0, 2113922820.0]",Y
4,NLP,5471114e37448bea2457b74894b1ecb92bbcfdf6,From Pretraining Data to Language Models to Do...,https://www.semanticscholar.org/paper/5471114e...,Annual Meeting of the Association for Computat...,Conference,Language models (LMs) are pretrained on divers...,2023,72,ArXiv,,"[2114887261.0, 50487261.0, 2169159066.0, 20735...",2.114887e+09,Tallinn,"[2235966.0, 8491577.0, 47610941.0]",Y


In [27]:
df_papers.to_csv('../aryan_data/papers_info.csv', index=False)